# Segmenting and Clustering Neighborhoods in Toronto (part 3/3)

In [1]:
#!conda install -c conda-forge beautifulsoup4 --yes # Install Beautiful Soup4 package for Web Scraping
#!conda install -c conda-forge html5lib --yes # Install html5lib package for HTML5 parsing
import numpy as np # library to handle data in a vectorized manner

from bs4 import BeautifulSoup # Tool for web scraping
import html5lib # HTML5 parser to parse the Wikipedia page
import requests # To download the HTML resource we want to scrap to our local server

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Download and Explore the data
Toronto has a total of 11 assigned boroughs and 103 postal codes (each postal code can encompass one or more neighborhoods). In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the boroughs, the postal codes and the neighborhoods. We will scrape the following Wikipedia page to collect this information: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### Load and explore the data
First, let's download the HTML page content to be scraped

In [2]:
html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text # Fetch the html content

Notice that all the relevant data is inside a **div** whose _id_ is 'mw-content-text'. Let's get first **table** tag inside that **div** using BeatifulSoup package for Web Scraping

In [3]:
soup = BeautifulSoup(html, "html5lib") # Using a HTML5 parser
table = soup.find('div', id='mw-content-text').table

Every row of the table (including the header) is inside the **tr** tag, so now we will store all the **tr** tags in a list

In [4]:
rows = table.find_all('tr') # List of all the rows in the Toronto Postal Code table

The next task is transforming this data DOM elements into a pandas dataframe. So let's start by creating an empty dataframe.

In [5]:
# define the dataframe columns
colum_names = ['PostalCode', 'Borough', 'Neighborhood']

# instantiate the dataframe
df = pd.DataFrame(columns = colum_names) 

Then let's loop through the rows and fill the dataframe one row at a time.

In [6]:
# Starting from the second row since the first row contains the column header names
for tr in rows[1:]:
    row = {}
    for i, cell in enumerate(tr.find_all('td')): # Loop through every cell in the row
        row[colum_names[i]] = cell.text.strip() # Fill every cell with the content of the td tag

    df = df.append(row, ignore_index=True)

### Process and Clean the DataFrame
Only process the cells that have an assigned a borough. Ignore cells with a borough that is Not assigned.

In [7]:
df = df[df['Borough'] != 'Not assigned']

Combine the neighborhoods that belong to a given postal code area into one row with the neighborhoods separated with a comma


In [8]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()

If a cell has a Borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [9]:
cond = df['Neighborhood'] == 'Not assigned'
df.loc[cond, 'Neighborhood'] = df.loc[cond, 'Borough']

Print the number of rows of the final dataframe

In [10]:
df.shape

(103, 3)

### Adding Latitude and Longitude information
Let's download geospatial information and save it as a CSV file called geo_coordinates.csv

In [11]:
!wget -q -O 'geo_coordinates.csv' http://cocl.us/Geospatial_data

Now that the data is downloaded, let's read it into a *pandas* dataframe.

In [12]:
geo_coor_df = pd.read_csv('geo_coordinates.csv')

Now let's add the spatial coordinates to the previous dataframe

In [13]:
# Set the primary key in both dataframe to be the Postal Code, and then make the inner join
df_toronto = df.set_index('PostalCode').join(geo_coor_df.set_index('Postal Code'), how = 'inner').reset_index()
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Checking dimensions...

In [14]:
df_toronto.shape

(103, 5)

### Create a map of Toronto with neighborhoods superimposed on top

In [15]:
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Use geopy library to get the latitude and longitude values of Toronto

In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, postal_code, borough, neighborhoods in zip(df_toronto['Latitude'], 
                                                         df_toronto['Longitude'], 
                                                         df_toronto['PostalCode'], 
                                                         df_toronto['Borough'], 
                                                         df_toronto['Neighborhood']):
    label = '{}, {}, {}'.format(neighborhoods, postal_code, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 2. Explore Neighborhoods in Toronto

#### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = '2KK1DY5UHRPGZUJSUYQWKVLDHQYQJPAZ114R5DTIA1RDR15K' 
CLIENT_SECRET = 'TDQE4UVYHJMB4KMZW0MFYRWXW3W5Y4SJX5VBTV4CMJ4IUNFN'
VERSION = '20181108' # Foursquare API version

#### Let's create a function to get the top venues that are in each postal code in Toronto within a certain radius

In [19]:
def getNearbyVenues(postal_codes, latitudes, longitudes, radius=500, limit=10):
    venues_list=[]
    for postal_code, lat, lng in zip(postal_codes, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postal_code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Running the above function on each postal code and creating a new dataframe called *toronto_venues*

In [20]:
# Default values for radius (500 meters) and limit (10 venues maximum)
toronto_venues = getNearbyVenues(postal_codes = df_toronto['PostalCode'],
                                   latitudes = df_toronto['Latitude'],
                                   longitudes = df_toronto['Longitude'])

Let's check the size of the resulting dataframe

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(693, 7)


,PostalCode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Let's check how many venues were returned for each postal code

In [23]:
toronto_venues.groupby('PostalCode').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,1,1,1,1,1,1
M1C,2,2,2,2,2,2
M1E,6,6,6,6,6,6
M1G,4,4,4,4,4,4
M1H,7,7,7,7,7,7
M1J,3,3,3,3,3,3
M1K,7,7,7,7,7,7
M1L,10,10,10,10,10,10
M1M,2,2,2,2,2,2


Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 178 uniques categories.


## 3. Analyze Each Postal Code

In [25]:
# one hot encoding for venues
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal codes column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move postal code column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Stadium,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,Fraternity House,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motel,Movie Theater,Museum,Music Venue,Neighborhood,Noodle House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tea Room,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M1C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Next, let's group rows by postal codes and by taking the mean of the frequency of occurrence of each category

In [26]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.head()

,PostalCode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Stadium,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,Fraternity House,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motel,Movie Theater,Museum,Music Venue,Neighborhood,Noodle House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tea Room,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Let's confirm the size

In [27]:
toronto_grouped.shape

(99, 179)

Originally there were 103 postal codes. That means there are 4 postal codes for which Foursquare did not return any venue within 500 meters

### Building a DataFrame with the top 10 venues
First, let's write a function to sort the venues in descending order.

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each postal code

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pc_venues_sorted = pd.DataFrame(columns=columns)
pc_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    pc_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

pc_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Yoga Studio,Empanada Restaurant,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
1,M1C,Golf Course,Bar,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
2,M1E,Electronics Store,Pizza Place,Breakfast Spot,Medical Center,Rental Car Location,Mexican Restaurant,Yoga Studio,Deli / Bodega,Discount Store,Diner
3,M1G,Coffee Shop,Korean Restaurant,Convenience Store,Department Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
4,M1H,Athletics & Sports,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant,Bank,Bakery,Department Store,Eastern European Restaurant,Drugstore


## 4. Cluster Postal Codes

In [30]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Run *k*-means to cluster the postal codes into 3 clusters. I chose this number because if I run the algorithm with 4 and 5, the 4th and 5th cluster will contain 1 and 2 points respectively. So 3 is OK because I get a representative number of points in each group.

In [31]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 0,
       1, 1, 2, 2, 0, 1, 1, 1, 2, 1, 1, 2, 2, 1, 0, 1, 1, 2, 0, 1, 2, 2, 0,
       2, 0, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       0, 1, 0, 1, 1, 1, 2, 0, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 2, 0, 1, 1], dtype=int32)

Now let's create a complete dataframe that includes the cluster as well as the top 10 venues for each postal code, with latitude, longitude, neighborhoods and boroughs.

In [32]:
toronto_merged = pc_venues_sorted
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_merged with df_toronto to add latitude/longitude, neighborhoods and boroughs
toronto_merged = toronto_merged.join(df_toronto.set_index('PostalCode'), on='PostalCode')

toronto_merged.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Borough,Neighborhood,Latitude,Longitude
0,M1B,Fast Food Restaurant,Yoga Studio,Empanada Restaurant,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,1,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Golf Course,Bar,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Electronics Store,Pizza Place,Breakfast Spot,Medical Center,Rental Car Location,Mexican Restaurant,Yoga Studio,Deli / Bodega,Discount Store,Diner,1,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Coffee Shop,Korean Restaurant,Convenience Store,Department Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,2,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Athletics & Sports,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant,Bank,Bakery,Department Store,Eastern European Restaurant,Drugstore,1,Scarborough,Cedarbrae,43.773136,-79.239476


Finally, let's visualize the resulting clusters

In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, postal_code, borough, neighborhoods, cluster in zip(toronto_merged['Latitude'], 
                                                                  toronto_merged['Longitude'], 
                                                                  toronto_merged['PostalCode'],
                                                                  toronto_merged['Borough'],
                                                                  toronto_merged['Neighborhood'],
                                                                  toronto_merged['Cluster Labels']):
    label = '{}, {}, {}. Cluster {}'.format(neighborhoods, postal_code, borough, cluster + 1)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, let's examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

#### Cluster 1

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0, 13, 14, 1, 2, 3, 4, 5]]]

,PostalCode,Neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,M1V,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,Park,Coffee Shop,Playground,Dance Studio,Drugstore
20,M2P,York Mills West,43.752758,Electronics Store,Park,Bank,Deli / Bodega,Eastern European Restaurant
22,M3A,Parkwoods,43.753259,Bus Stop,Park,Fast Food Restaurant,Food & Drink Shop,Dance Studio
27,M3K,"CFB Toronto, Downsview East",43.737473,Electronics Store,Airport,Park,Bus Stop,Deli / Bodega
37,M4J,East Toronto,43.685347,Coffee Shop,Park,Convenience Store,Deli / Bodega,Eastern European Restaurant
41,M4N,Lawrence Park,43.728020,Lake,Park,Dim Sum Restaurant,Swim School,Bus Line
45,M4T,"Moore Park, Summerhill East",43.689574,Tennis Court,Park,Playground,Curling Ice,Dog Run
47,M4W,Rosedale,43.679563,Park,Playground,Trail,Yoga Studio,Curling Ice
69,M6B,Glencairn,43.709577,Park,Japanese Restaurant,Pub,Asian Restaurant,Metro Station
71,M6E,Caledonia-Fairbanks,43.689026,Park,Pharmacy,Women's Store,Market,Fast Food Restaurant


Cluster 1 represents mostly locations nearby parks

#### Cluster 2

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0, 13, 14, 1, 2, 3, 4, 5]]]

,PostalCode,Neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,"Rouge, Malvern",43.806686,Fast Food Restaurant,Yoga Studio,Empanada Restaurant,Eastern European Restaurant,Drugstore
1,M1C,"Highland Creek, Rouge Hill, Port Union",43.784535,Golf Course,Bar,Yoga Studio,Deli / Bodega,Eastern European Restaurant
2,M1E,"Guildwood, Morningside, West Hill",43.763573,Electronics Store,Pizza Place,Breakfast Spot,Medical Center,Rental Car Location
4,M1H,Cedarbrae,43.773136,Athletics & Sports,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant
5,M1J,Scarborough Village,43.744734,Playground,Jewelry Store,Health & Beauty Service,Yoga Studio,Deli / Bodega
7,M1L,"Clairlea, Golden Mile, Oakridge",43.711112,Bus Line,Bakery,Soccer Field,Ice Cream Shop,Metro Station
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,Motel,American Restaurant,Yoga Studio,Eastern European Restaurant,Drugstore
9,M1N,"Birch Cliff, Cliffside West",43.692657,College Stadium,Skating Rink,General Entertainment,Café,Deli / Bodega
10,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Latin American Restaurant
11,M1R,"Maryvale, Wexford",43.750072,Middle Eastern Restaurant,Breakfast Spot,Auto Garage,Shopping Mall,Sandwich Place


Cluster 2 represents mostly locations nearby gastro places: restaurants, grocery stores, pizza, etc

#### Cluster 3

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0, 13, 14, 1, 2, 3, 4, 5]]]

,PostalCode,Neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,M1G,Woburn,43.770992,Coffee Shop,Korean Restaurant,Convenience Store,Department Store,Eastern European Restaurant
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",43.727929,Discount Store,Coffee Shop,Chinese Restaurant,Train Station,Hobby Shop
21,M2R,Willowdale West,43.782736,Pharmacy,Pizza Place,Butcher,Coffee Shop,Dance Studio
25,M3H,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,Coffee Shop,Restaurant,Bridal Shop,Sandwich Place,Diner
26,M3J,"Northwood Park, York University",43.767980,Coffee Shop,Eastern European Restaurant,Massage Studio,Bar,Department Store
31,M4A,Victoria Village,43.725882,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Dance Studio
34,M4E,The Beaches,43.676357,Coffee Shop,Neighborhood,Trail,Pub,Dance Studio
35,M4G,Leaside,43.709060,Coffee Shop,Sushi Restaurant,Gastropub,Bike Shop,Gym
40,M4M,Studio District,43.659526,Coffee Shop,Café,Breakfast Spot,Sandwich Place,Fish Market
43,M4R,North Toronto West,43.715383,Sporting Goods Shop,Yoga Studio,Chinese Restaurant,Mexican Restaurant,Dessert Shop


Cluster 3 represents mostly locations nearby coffe shops